In [1]:
import glob
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
from pandas.api.types import is_numeric_dtype
from collections import Counter,defaultdict

In [2]:
def summary_stats(dat, key_s):
    b_data = []
    for col in key_s:
        nans = np.count_nonzero(pd.isnull(dat[col]))
        dist_val = len(pd.unique(dat[col].dropna()))
        Total_val = len(dat[col])
        #print(Total_val)
        mean = 0
        std_dev = 0
        var = 0
        min_val = 0
        max_val = 0
        if is_numeric_dtype(dat[col]):
            mean = np.mean(dat[col])
            
            if pd.isnull(mean):
                mean = 0
                std_dev = 0
                #var = 0
                min_val = 0
                max_val = 0
                
            else:    
                std_dev = np.std(dat[col])
                var = np.var(dat[col])
                min_val = float(np.min(dat[col]))
                max_val = float(np.max(dat[col]))
        b_data.append([Total_val, nans, dist_val, mean, std_dev, min_val, max_val])
    return b_data

def castability_feature(dat, column_names):
    castability_list = []
    #make sure the value you are avaluating is not nan
    for keys in column_names:
        #print(keys)
        i = 0
        while pd.isnull(dat[keys][i]):
            i += 1
            if i > len(dat[keys]) - 2:
                break
        #if type is string try casting
        if dat[keys][i].__class__.__name__ == 'str':
            try:
                castability = str(type(eval(dat[keys][i])))
                castability_list.append(1)
            except:
                castability_list.append(0)
        else:
            castability_list.append(0)
    return castability_list  

def get_class_type(dat, column_names):
    as_read = []
    master_key_dictionary =  master_key()
    for keys in column_names:
        
        #make sure the value you are avaluating is not nan
        i = 0
        while pd.isnull(dat[keys][i]):
            i += 1
            if i > len(dat[keys]) - 2:
                break
        val = -1
        type_pyth = dat[keys][i].__class__.__name__
        for tipe in master_key_dictionary:
            if tipe in type_pyth:
                val = master_key_dictionary[tipe]
        as_read.append(val)
    return as_read

def master_key():
    master_key_dic = defaultdict(int)
    master_key_dic['str'] = 0
    master_key_dic['float'] = 1
    master_key_dic['int'] = 1
    return master_key_dic

def numeric_extraction(dat,column_names):
    #0 no , 1 yes
    numeric_extraction_list = []
    #make sure the value you are avaluating is not nan
    for keys in column_names:
        i = 0
        while pd.isnull(dat[keys][i]):
            i += 1
            if i > len(dat[keys]) - 2:
                break
        val = 0
            
        if dat[keys][i].__class__.__name__ == 'str':
            #print('yes')
            #check whether any number can be extracted
            try:
                #it will faile when you have no numbers or if you have two numbers seperated by space
                float(re.sub('[^0-9\. ]', ' ',dat[keys][i]))
                #print('yes')
                val = 1
            except:
                pass
            
        numeric_extraction_list.append(val)
    
    return numeric_extraction_list


def val_length(dat,column_names):
    val = []
    for keys in column_names:
        i = 0
        while pd.isnull(dat[keys][i]):
            i += 1
            if i > len(dat[keys]) - 2:
                break
        val.append(len(str(dat[keys][i]).split()))
    return val      

In [3]:
def get_sample(dat, key_s):
    rand = []
    for name in keys:
        rand_sample = list(pd.unique(dat[name]))
        rand_sample = rand_sample[:5]
        while len(rand_sample) < 5:
            rand_sample.append(list(pd.unique(dat[name]))[np.random.randint(len(list(pd.unique(dat[name]))))])
        rand.append(rand_sample[:5])
    return rand
        

In [4]:
# read golden data
csv_path = glob.glob('./data_set/golden_dataset/*.csv')
stats = []
attribute_name = []
sample = []
csv_names = []
id_value = []
i = 0
url = list(pd.read_csv('./data_set/golden_meta_data/golden_url.csv', header = None)[0])
url_list = []
castability = []
class_type = []
number_extraction = []
value_length = []

In [5]:
for path_name in tqdm(csv_path):
    i = i+1
    df = pd.read_csv(path_name,encoding = 'latin1',lineterminator='\n')
    keys = list(df.keys())
    #get features
    attribute_name.extend(keys)
    stats.extend(summary_stats(df, keys))
    sample.extend(get_sample(df,keys))
    csv_names.extend([path_name.split('/')[-1] for j in range(len(keys))])
    id_value.extend(["{0:0>5}".format(i) for j in range(len(keys))])
    url_list.extend([url[i-1] for j in range(len(keys))])
    castability.extend(castability_feature(df, keys))
    class_type.extend(get_class_type(df, keys))
    number_extraction.extend(numeric_extraction(df, keys))
    value_length.extend(val_length(df, keys))
label_dict = {1:'Usable directly numeric', 2:'Usable with extraction', 3:'Usable directly categorical', 4:'Unusable', 5:'Context dependent'}
## labels
### 1. Usable directly numeric
### 2. Usable with extraction 
### 3. Usable directly categorical
### 4. Unusable

100%|██████████| 32/32 [00:04<00:00,  7.53it/s]


# Adding y from Rule_check.csv

In [6]:
golden_rule_check = pd.read_csv('/Users/prem/Desktop/CSE 291A/data_set/golden_meta_data/Rule_check.csv')
d2 = dict((v, k) for k, v in label_dict.items())
y = [d2[golden_rule_check['Label'].loc[i]] for i in range(len(golden_rule_check['Label']))]

In [7]:
#create golden csv
csv_names = ['Record_id', 'label', 'Attribute_name','Total_val', 'Num of nans', 'num of dist_val', 'mean', 'std_dev', 'min_val', 'max_val','sample_1', 'sample_2', 'sample_3','sample_4','sample_5']
golden_data = pd.DataFrame(columns = csv_names)
for i in tqdm(range(len(y))):
    val_append = [id_value[i]]
    val_append.append(label_dict[y[i]])
    val_append.append(attribute_name[i])
    val_append.extend(stats[i])
    val_append.extend(sample[i])
    golden_data.loc[i] = val_append
golden_data = golden_data.replace({r'\r': ''}, regex=True)
golden_data.to_csv('golden_data.csv', index = False)

100%|██████████| 489/489 [00:03<00:00, 141.75it/s]


In [8]:
#create meta data for golden csv
csv_names = ['Dataset_name', 'URL', 'CSV_filename','RecordID']
golden_data_meta = pd.DataFrame(columns = csv_names)
i = 0
for path_name in tqdm(csv_path):
    val_append = [path_name.split('/')[-1]]
    val_append.append(url[i])
    val_append.append(path_name.split('/')[-1])
    val_append.append(list(np.unique(id_value))[i])
    golden_data_meta.loc[i] = val_append
    i = i+1
golden_data_meta.to_csv('golden_data_meta.csv', index = False)

100%|██████████| 32/32 [00:00<00:00, 240.46it/s]


In [9]:
#create data for random_forest
csv_names = ['Record_id', 'y', 'Attribute_name','Total_val', 'Num of nans', 'num of dist_val', 'mean', 'std_dev', 'min_val', 'max_val','val_castability', 'val_extractability','val_length', 'val_type']
golden_data = pd.DataFrame(columns = csv_names)
for i in tqdm(range(len(y))):
    val_append = [id_value[i]]
    val_append.append(y[i])
    val_append.append(attribute_name[i])
    val_append.extend(stats[i])
    val_append.append(castability[i])
    val_append.append(number_extraction[i])
    val_append.append(value_length[i])
    val_append.append(class_type[i])
    golden_data.loc[i] = val_append
golden_data = golden_data.replace({r'\r': ''}, regex=True)
golden_data.to_csv('golden_data_Random_Forest.csv', index = False)

100%|██████████| 489/489 [00:03<00:00, 129.93it/s]


In [10]:
# read data and initiate values we need
stats = []
attribute_name = []
sample = []
csv_names = []
id_value = []
i = 0
url = list(pd.read_csv('./data_set/meta_data/meta_data.csv', header = None)[1])
csv_list = list(pd.read_csv('./data_set/meta_data/meta_data.csv', header = None)[0])
url_list = []
castability = []
class_type = []
number_extraction = []
value_length = []

In [11]:
for path_name in tqdm(csv_list):
    i = i+1
    df = pd.read_csv(os.path.join('./data_set/dataset/',path_name),encoding = 'latin1',lineterminator='\n')
    keys = list(df.keys())
    #get features
    attribute_name.extend(keys)
    stats.extend(summary_stats(df, keys))
    sample.extend(get_sample(df,keys))
    csv_names.extend([path_name.split('/')[-1] for j in range(len(keys))])
    id_value.extend(["{0:0>5}".format(i+len(golden_data_meta)) for j in range(len(keys))])
    url_list.extend([url[i-1] for j in range(len(keys))])
    castability.extend(castability_feature(df, keys))
    class_type.extend(get_class_type(df, keys))
    number_extraction.extend(numeric_extraction(df, keys))
    value_length.extend(val_length(df, keys))

  5%|▍         | 16/329 [00:35<11:33,  2.21s/it]/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 28%|██▊       | 92/329 [01:46<04:33,  1.15s/it]/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 29%|██▊       | 94/329 [01:58<04:56,  1.26s/it]/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,19,137,138,250) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 30%|██▉       | 98/329 [02:03<04:51,  1.26s/it]/usr/local/lib/python3.6/site-packages/IPython/core/int

In [12]:
#create csv
csv_names = ['Record_id', 'Attribute_name','Total_val', 'Num of nans', 'num of dist_val', 'mean', 'std_dev', 'min_val', 'max_val','sample_1', 'sample_2', 'sample_3','sample_4','sample_5']
data = pd.DataFrame(columns = csv_names)
for i in tqdm(range(len(attribute_name))):
    val_append = [id_value[i]]
    val_append.append(attribute_name[i])
    val_append.extend(stats[i])
    val_append.extend(sample[i])
    data.loc[i] = val_append

100%|██████████| 10741/10741 [02:05<00:00, 85.59it/s]


In [13]:
data = data.replace({r'\r': ''}, regex=True)
data.to_csv('data.csv', index = False)

In [14]:
#create meta data for csv
csv_names = ['Dataset_name', 'URL', 'CSV_filename','RecordID']
data_meta = pd.DataFrame(columns = csv_names)
i = 0
for i in tqdm(range(len(csv_list))):
    val_append = [csv_list[i]]
    val_append.append(url[i])
    val_append.append(csv_list[i])
    val_append.append(list(np.unique(id_value))[i])
    data_meta.loc[i] = val_append
    i = i+1
data_meta.to_csv('data_meta.csv',sep=',', index = False)

100%|██████████| 329/329 [00:02<00:00, 159.41it/s]


In [15]:
golden_rule_check = pd.read_csv('/Users/prem/Desktop/CSE 291A/data_set/golden_meta_data/Rule_check.csv')

In [16]:
d2 = dict((v, k) for k, v in label_dict.items())
Counter([0 if y[i]-d2[golden_rule_check['Label'].loc[i]] == 0 else i for i in range(len(golden_rule_check['Label']))])

Counter({0: 489})